This notebook consists of

(i) using openAI_4o API to perform commentary generation

In [1]:
#@title Codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks


In [2]:
#@title Pip install requirements.txt
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
!pip install openai

In [4]:
#@title Add Cuda Device
import torch
device = 0 if torch.cuda.is_available() else -1  # device 0 for the first GPU
device

0

In [5]:
#@title Import necessary libraries
import os
import logging
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd

import openai
import logging

import cv2
import base64

from tqdm import tqdm
#from video_processor import extract_encode_frames, create_message
#from openai_client import get_commentary_for_frames

In [11]:
import glob

len([path for path in glob.glob('./data/raw/NSVA_Data/NSVA_Video6/*.mp4')])

223

In [ ]:
'''
video_dir = './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx'
abs_video_dir = Path(video_dir).resolve()
for path in abs_video_dir.glob('*.mp4'):
    print(path)
'''

In [9]:
game = '0021800013-dal-vs-phx'
metadata_file = f'./data/processed/final_results.csv'
pd.read_csv(metadata_file).head()

,Unnamed: 0,video_path,video_id,pred_actionType1,predprob_actionType1,pred_actionType2,predprob_actionType2
0,0,data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-...,10978,Missed Shot,0.813380,Made Shot,0.092498
1,1,data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-...,10980,Missed Shot,0.770581,Made Shot,0.143033
2,2,data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-...,10982,Made Shot,0.645901,Missed Shot,0.184492
3,3,data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-...,10983,Made Shot,0.805227,Foul,0.111543
4,4,data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-...,10985,Foul,0.912886,Made Shot,0.038231


In [18]:
#@title Step 1) Useful Functions


def main():
    #load_dotenv()

    frame_interval = 60
    video_dir = './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx'
    text_dir = './data/text/GPT4o/0021800013-dal-vs-phx_commentary_results2.csv'
    #text_dir = 'commentary_results.csv'
    #api_key = os.getenv('OPENAI_API_KEY')

    from google.colab import userdata
    api_key = userdata.get('hoopstalk')
    #print('check ur api key',userdata.get('hoopstalk'))

    if not api_key:
        logging.error("API key not found. Make sure to set the OPENAI_API_KEY environment variable.")
        return

    # Convert the directory string to a Path object
    abs_video_dir = Path(video_dir).resolve()
    logging.info(f"Absolute video directory: {abs_video_dir}")

    results = []
    commentary_buffer = []  # Initialize a buffer to store the last 10 lines of commentary

    # Use glob to find all .mp4 files in the directory
    video_files = glob.glob('./data/raw/NSVA_Data/NSVA_Video6/*.mp4')#[:2] #take first two

    for video_file in tqdm(video_files, desc="Processing Videos"):
    #for video_file in glob.glob('./data/raw/NSVA_Data/NSVA_Video/*.mp4'):
        print('mp4',video_file)
        logging.info(f"Processing video: {video_file}")
        base64_frames = extract_encode_frames(video_file, frame_interval)
        messages = create_message(base64_frames, frame_interval, "\n".join(commentary_buffer))
        commentary = get_commentary_for_frames(api_key, messages)

        if commentary:
            #results.append({'video_file': video_file.name, 'commentary': commentary})
            results.append({'video_file': video_file, 'commentary': commentary})
            commentary_buffer.extend(commentary.split("\n"))  # Update the buffer with new lines
            commentary_buffer = commentary_buffer[-10:]  # Keep only the last 10 lines
        else:
            logging.error(f"Failed to get commentary for {video_file.name}")

    # Save results to CSV
    print('results',results)
    df = pd.DataFrame(results)
    print(text_dir)
    df.to_csv(text_dir, index=False)
    logging.info(f"Commentary saved to {text_dir}")

def get_commentary_for_frames(api_key, messages, temperature=0.7, max_tokens=300, model="gpt-4o"):
    openai.api_key = api_key

    try:
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        # logging.info(f"GPT Response: {response.choices[0].message.content}")
        return response.choices[0].message.content
    except openai.error.OpenAIError as e:
        logging.error(f"OpenAI API error: {e}")
        return None

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_encode_frames(video_path, frame_interval=50):
    video_path = str(video_path).replace('\\', '/')
    video = cv2.VideoCapture(video_path)

    base64_frames = []
    frame_count = 0
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        if frame_count % frame_interval == 0:
            _, buffer = cv2.imencode(".jpg", frame)
            base64_frames.append(base64.b64encode(buffer).decode("utf-8"))
        frame_count += 1

    video.release()

    return base64_frames

def create_message(encoded_frames, frame_interval=50, previous_commentary=""):
    content = [
        "You are an NBA commentator. Here are frames from a recent NBA game. Your task is to provide a brief and engaging commentary summarizing the key events depicted in these frames. Please maintain a flow and continuity with the previous commentary provided. Ensure that the events described are logically sequenced and contextually appropriate for different parts of the game:",
        *map(lambda x: {"image": x, "resize": 768}, encoded_frames[0::frame_interval]),
        f"Previous commentary:\n{previous_commentary}",
        "Consider the following types of events to help structure your commentary:",
        """
        1. Play Start: Describe the beginning of an offensive or defensive play.
        2. Shot Attempt: [Player] attempts a [shot type] from [distance].
        3. Rebound: [Player] grabs the rebound.
        4. Successful Shot: [Player] scores a [shot type] from [distance].
        5. Assist: Assist by [Player].
        6. Foul: [Player] commits a [foul type].
        7. Turnover: [Player] loses the ball.
        8. Significant Moment: Describe any significant moment like a game-changing play or a remarkable defensive action.
        9. End of Period: End of [period number].
        Combine these elements to provide a cohesive and engaging summary of the key events depicted in these frames in approximately two sentences.
        """
    ]

    messages = [{"role": "user", "content": content}]

    return messages


In [19]:
logging.basicConfig(level=logging.INFO)
main()

Processing Videos:   0%|          | 0/223 [00:00<?, ?it/s]

mp4 ./data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_10977.mp4


Processing Videos:   0%|          | 1/223 [00:08<31:13,  8.44s/it]

mp4 ./data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_10978.mp4


Processing Videos:   1%|          | 2/223 [00:15<27:27,  7.45s/it]

...

mp4 ./data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_11287.mp4


Processing Videos: 100%|█████████▉| 222/223 [29:28<00:08,  8.53s/it]

mp4 ./data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_11289.mp4


Processing Videos: 100%|██████████| 223/223 [29:38<00:00,  7.97s/it]

results [{'video_file': './data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_10977.mp4', 'commentary': "The game is underway as the teams prepare for the opening tip-off at the Talking Stick Resort Arena. The jump ball is up, and it's secured by the team in white, setting the stage for an exciting start to the game."}, {'video_file': './data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_10978.mp4', 'commentary': '"And here we go, the Mavericks have the ball and are setting up their first play of the game. The pass goes inside to the big man, who is looking to make an early impact against a tightly-knit Suns defense. The clock is ticking down, let\'s see if they can capitalize on this possession to get the first points on the board."'}, {'video_file': './data/raw/NSVA_Data/NSVA_Video6/0021800013-dal-vs-phx_10980.mp4', 'commentary': '"Booker initiates the play for the Suns, dribbling at the top of the key as the Mavericks\' defense closes in. He spots Ayton in the post, who receives